##  main.dbの作成

In [2]:
import requests
import sqlite3

# URLからJSONデータを取得
url = "http://www.jma.go.jp/bosai/common/const/area.json"
response = requests.get(url)

# レスポンスのエンコーディングをUTF-8に設定
response.encoding = 'utf-8'

# JSONデータをPythonの辞書として解析
area_json = response.json()

# "centers" キー内のデータを取得
centers = area_json.get("centers", {})

# SQLiteデータベース接続
db_filename = 'area_info.db'
conn = sqlite3.connect(db_filename)
cursor = conn.cursor()

# テーブルの作成（もしテーブルが存在しない場合）
cursor.execute("""
CREATE TABLE IF NOT EXISTS area_info (
    id INTEGER PRIMARY KEY,
    area_code TEXT,
    area_name TEXT,
    en_name TEXT,
    office_name TEXT,
    children_code TEXT
)
""")

# idの初期化
id_counter = 1

# 各エリア情報をSQLiteデータベースに挿入
for area_code, area_info in centers.items():
    name = area_info.get("name", "N/A")
    enName = area_info.get("enName", "N/A")
    officeName = area_info.get("officeName", "N/A")
    children = area_info.get("children", [])

    # 親エリアの情報をSQLiteに書き込む
    if children:
        for region_code in children:
            # 子エリアIDを挿入
            cursor.execute("""
            INSERT INTO area_info (id, area_code, area_name, en_name, office_name, children_code)
            VALUES (?, ?, ?, ?, ?, ?)
            """, (id_counter, area_code, name, enName, officeName, region_code))
            id_counter += 1
    else:
        # 子エリアがいない場合
        cursor.execute("""
        INSERT INTO area_info (id, area_code, area_name, en_name, office_name, children_code)
        VALUES (?, ?, ?, ?, ?, ?)
        """, (id_counter, area_code, name, enName, officeName, 'N/A'))
        id_counter += 1

# コミットして変更を保存
conn.commit()

# データベース接続を閉じる
conn.close()

IntegrityError: UNIQUE constraint failed: area_info.id

In [7]:
import sqlite3
import requests

# 新しいデータベースのパスと名前
db_filename = 'weather_areas.db'

# 新しいデータベースに接続（存在しない場合は作成されます）
con = sqlite3.connect(db_filename)
cur = con.cursor()

# area_info.dbからデータを取得して、weather_areas.dbに保存する準備
area_info_db = 'area_info.db'
con_area_info = sqlite3.connect(area_info_db)
cur_area_info = con_area_info.cursor()

# area_infoテーブルからデータを取得
sql_select = "SELECT * FROM area_info;"
cur_area_info.execute(sql_select)

# area_infoテーブルをweather_areas.dbにコピー（テーブルが存在しない場合は作成）
cur.execute("""
CREATE TABLE IF NOT EXISTS area_info (
    id INTEGER PRIMARY KEY,
    area_code TEXT,
    area_name TEXT,
    en_name TEXT,
    office_name TEXT,
    children_code TEXT
)
""")

# area_infoからデータを1行ずつ取得し、weather_areas.dbのarea_infoテーブルに挿入
for row in cur_area_info:
    id, area_code, area_name, en_name, office_name, children_code = row
    cur.execute("""
    INSERT INTO area_info (id, area_code, area_name, en_name, office_name, children_code)
    VALUES (?, ?, ?, ?, ?, ?)
    """, (id, area_code, area_name, en_name, office_name, children_code))

# weather_forecast_areasテーブルを作成
cur.execute("""
CREATE TABLE IF NOT EXISTS weather_forecast_areas (
    id INTEGER,
    children_code TEXT,
    area_name TEXT,
    area_code TEXT,
    date TEXT,
    weather_codes TEXT,
    weathers TEXT,
    winds TEXT,
    waves TEXT
)
""")

# area_info.dbから取得したデータを使って、weather_forecast_areasテーブルに予報データを挿入
for row in cur_area_info:
    id, area_code, area_name, en_name, office_name, children_code = row

    # children_codeを使ってURLを作成
    url = f"https://www.jma.go.jp/bosai/forecast/data/forecast/{children_code}.json"
    
    try:
        # URLからJSONデータを取得
        response = requests.get(url)
        response.raise_for_status()  # レスポンスが成功でない場合、例外を発生

        # JSONデータを解析
        data = response.json()

        # 時刻（timeDefines）の情報を取得
        time_defines = data[0]['timeSeries'][0]['timeDefines']
        
        # 各エリアの情報を取得
        areas = data[0]['timeSeries'][0]['areas']

        # 各エリアの情報をループ処理
        for area in areas:
            area_name = area['area']['name']
            area_code = area['area']['code']
            weather_codes = area.get('weatherCodes', ['なし'])
            weathers = area.get('weathers', ['なし'])
            winds = area.get('winds', ['なし'])
            waves = area.get('waves', ['なし'])

            # データを1行ずつDBに挿入
            for i in range(len(time_defines)):
                row_data = (
                    id,  # ID
                    children_code,  # Children Code
                    area_name,  # Area Name
                    area_code,  # Area Code
                    time_defines[i],  # Date
                    weather_codes[i] if i < len(weather_codes) else 'なし',  # Weather Codes
                    weathers[i] if i < len(weathers) else 'なし',  # Weathers
                    winds[i] if i < len(winds) else 'なし',  # Winds
                    waves[i] if i < len(waves) else 'なし'  # Waves
                )
                # DBにデータを挿入
                cur.execute("""
                INSERT INTO weather_forecast_areas (
                    id, children_code, area_name, area_code, date, weather_codes, weathers, winds, waves
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                """, row_data)

    except requests.exceptions.RequestException as e:
        # リクエストエラーが発生した場合
        print(f"Error fetching data from URL {url}: {e}")

# コミットして変更を保存
con.commit()

# 作成されたテーブルの一覧を表示
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

print("作成されたテーブル一覧:")
for table in tables:
    table_name = table[0]
    if table_name in ["area_info", "weather_forecast_areas"]:
        print(f"テーブル名: {table_name}")
        
        # 各テーブルのデータを表示（最大10行のみ表示）
        cur.execute(f"SELECT * FROM {table_name} LIMIT 10;")
        rows = cur.fetchall()
        print(f"テーブル {table_name} のデータ:")
        for row in rows:
            print(row)
        print("-" * 50)

# DBへの接続を閉じる
con_area_info.close()
con.close()

print("データベースの作成とデータ挿入が完了しました。")


作成されたテーブル一覧:
テーブル名: area_info
テーブル area_info のデータ:
(1, '010100', '北海道地方', 'Hokkaido', '札幌管区気象台', '011000')
(2, '010100', '北海道地方', 'Hokkaido', '札幌管区気象台', '012000')
(3, '010100', '北海道地方', 'Hokkaido', '札幌管区気象台', '013000')
(4, '010100', '北海道地方', 'Hokkaido', '札幌管区気象台', '014030')
(5, '010100', '北海道地方', 'Hokkaido', '札幌管区気象台', '014100')
(6, '010100', '北海道地方', 'Hokkaido', '札幌管区気象台', '015000')
(7, '010100', '北海道地方', 'Hokkaido', '札幌管区気象台', '016000')
(8, '010100', '北海道地方', 'Hokkaido', '札幌管区気象台', '017000')
(9, '010200', '東北地方', 'Tohoku', '仙台管区気象台', '020000')
(10, '010200', '東北地方', 'Tohoku', '仙台管区気象台', '030000')
--------------------------------------------------
テーブル名: weather_forecast_areas
テーブル weather_forecast_areas のデータ:
--------------------------------------------------
データベースの作成とデータ挿入が完了しました。


In [3]:
import sqlite3

# データベースのパスと名前
db_filename = 'area_info.db'

# DBに接続
con = sqlite3.connect(db_filename)

# SQLを実行するためのカーソルオブジェクトを取得
cur = con.cursor()

# データを参照するSQL文
# 例: area_infoテーブルのすべてのデータを取得する
sql_select = "SELECT * FROM area_info;"

# SQLを実行
cur.execute(sql_select)

# データベースから取得したデータをループで表示
for row in cur:
    # 行データ(row)はタプルなので、アンパックして列データを取得
    id, area_code, area_name, en_name, office_name, children_code = row
    print(f"id: {id}, area_code: {area_code}, area_name: {area_name}, en_name: {en_name}, office_name: {office_name}, children_code: {children_code}")

# DBへの接続を閉じる
con.close()


id: 1, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 011000
id: 2, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 012000
id: 3, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 013000
id: 4, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 014030
id: 5, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 014100
id: 6, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 015000
id: 7, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 016000
id: 8, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 017000
id: 9, area_code: 010200, area_name: 東北地方, en_name: Tohoku, office_name: 仙台管区気象台, children_code: 020000
id: 10, area_code: 010200, area_name: 東北

##  地町村ごとの天気予報を取得する

In [1]:
import sqlite3
import requests

# データベースのパスと名前
db_filename = 'weather_forecast.db'

# 新しいデータベースに接続（存在しない場合は作成されます）
con = sqlite3.connect(db_filename)
cur = con.cursor()

# 既存のarea_info.dbからデータを取得
area_info_db = 'area_info.db'
con_area_info = sqlite3.connect(area_info_db)
cur_area_info = con_area_info.cursor()

# SQLを実行して、area_infoテーブルからデータを取得
sql_select = "SELECT * FROM area_info;"
cur_area_info.execute(sql_select)

# データベースから取得したデータをループで処理
for row in cur_area_info:
    # 行データ(row)はタプルなので、アンパックして列データを取得
    id, area_code, area_name, en_name, office_name, children_code = row
    print(f"id: {id}, area_code: {area_code}, area_name: {area_name}, en_name: {en_name}, office_name: {office_name}, children_code: {children_code}")

    # children_codeを使ってURLを作成
    url = f"https://www.jma.go.jp/bosai/forecast/data/forecast/{children_code}.json"
    
    try:
        # URLからJSONデータを取得
        response = requests.get(url)
        response.raise_for_status()  # レスポンスが成功でない場合、例外を発生

        # JSONデータを解析
        data = response.json()

        # 時刻（timeDefines）の情報を取得
        time_defines = data[0]['timeSeries'][0]['timeDefines']
        
        # 各エリアの情報を取得
        areas = data[0]['timeSeries'][0]['areas']  # ここでエリアのリストを取得

        # id と area_code 別のテーブル作成
        # テーブル名を "weather_forecast_areas_{area_code}_{id}" にする
        table_name = f"weather_forecast_areas_{area_code}_{id}"
        
        # 新しいテーブルを作成（area_codeとidをテーブル名にする）
        cur.execute(f"""
        CREATE TABLE IF NOT EXISTS "{table_name}" (
            id INTEGER,
            children_code TEXT,
            area_name TEXT,
            area_code TEXT,
            date TEXT,
            weather_codes TEXT,
            weathers TEXT,
            winds TEXT,
            waves TEXT
        )
        """)

        # 各エリアの情報をループ処理
        for area in areas:
            area_name = area['area']['name']
            area_code = area['area']['code']
            weather_codes = area.get('weatherCodes', ['なし'])
            weathers = area.get('weathers', ['なし'])
            winds = area.get('winds', ['なし'])
            waves = area.get('waves', ['なし'])

            # データを1行ずつDBに挿入
            for i in range(len(time_defines)):
                # 行データを準備
                row_data = (
                    id,  # ID
                    children_code,  # Children Code
                    area_name,  # Area Name
                    area_code,  # Area Code
                    time_defines[i],  # Date
                    weather_codes[i] if i < len(weather_codes) else 'なし',  # Weather Codes
                    weathers[i] if i < len(weathers) else 'なし',  # Weathers
                    winds[i] if i < len(winds) else 'なし',  # Winds
                    waves[i] if i < len(waves) else 'なし'  # Waves
                )
                # DBにデータを挿入（area_codeとidをテーブル名として使用）
                cur.execute(f"""
                INSERT INTO "{table_name}" (
                    id, children_code, area_name, area_code, date, weather_codes, weathers, winds, waves
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                """, row_data)

    except requests.exceptions.RequestException as e:
        # リクエストエラーが発生した場合
        print(f"Error fetching data from URL {url}: {e}")

# コミットして変更を保存
con.commit()

# 作成されたテーブルの一覧を表示
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

print("作成されたテーブル一覧:")
for table in tables:
    table_name = table[0]
    if table_name.startswith("weather_forecast_areas_"):
        print(f"テーブル名: {table_name}")
        
        # 各テーブルのデータを表示（最大10行のみ表示）
        cur.execute(f"SELECT * FROM {table_name} LIMIT 10;")
        rows = cur.fetchall()
        print(f"テーブル {table_name} のデータ:")
        for row in rows:
            print(row)
        print("-" * 50)

# DBへの接続を閉じる
con_area_info.close()
con.close()

print("データベースの作成とデータ挿入が完了しました。")


id: 1, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 011000
id: 2, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 012000
id: 3, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 013000
id: 4, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 014030
Error fetching data from URL https://www.jma.go.jp/bosai/forecast/data/forecast/014030.json: 404 Client Error: Not Found for url: https://www.jma.go.jp/bosai/forecast/data/forecast/014030.json
id: 5, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 014100
id: 6, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 015000
id: 7, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 016000
id: 8, area_code: 010100, area_name: 北海道地方, en_name: Hokka

##  一番見やすい形にしてみる

In [3]:
import sqlite3
import requests  # 修正：requestsのインポートのスペルを修正

# データベースのパスと名前
db_filename = 'weather_forecast_detail.db'

# 新しいデータベースに接続（存在しない場合は作成されます）
con = sqlite3.connect(db_filename)
cur = con.cursor()

# area_info.dbからデータを取得
area_info_db = 'area_info.db'
con_area_info = sqlite3.connect(area_info_db)
cur_area_info = con_area_info.cursor()

# SQLを実行して、area_infoテーブルからデータを取得 (DISTINCTを使用)
sql_select = "SELECT DISTINCT * FROM area_info;"  # 重複排除
cur_area_info.execute(sql_select)

# 取得したデータをループで処理
for row in cur_area_info:
    # 行データ(row)はタプルなので、アンパックにして列データを取得
    id, area_code, area_name, en_name, office_name, children_code = row
    print(f"id: {id}, area_code: {area_code}, area_name: {area_name}, en_name: {en_name}, office_name: {office_name}, children_code: {children_code}")

    # children_codeを使ってURLを作成
    url = f"https://www.jma.go.jp/bosai/forecast/data/forecast/{children_code}.json"
    
    try:
        # URLからJSONデータを取得
        response = requests.get(url)
        response.raise_for_status()  # 応答が成功しない場合、例外が発生
        
        # JSONデータを解析
        data = response.json()
        
        # 時刻（timeDefines）の情報を取得
        time_defines = data[0]['timeSeries'][0]['timeDefines']
        
        # 各エリアの情報を取得
        area = data[0]['timeSeries'][0]['areas']
        
        # idとarea_code別のテーブル作成
        table_name = f"weather_forecast_areas_{area_code}_{id}"
        
        # 新しいテーブルを作成（area_codeとidをテーブル名にする）
        cur.execute(f"""
        CREATE TABLE IF NOT EXISTS "{table_name}" (
            id INTEGER,
            children_code TEXT,
            area_name TEXT,
            area_code TEXT,
            date TEXT,
            weather_codes TEXT,
            weather TEXT,
            wind TEXT,
            wave TEXT,
            UNIQUE(id, children_code, date)  -- 重複を防ぐためにid, children_code, dateをユニーク制約
        )
        """)
        
        # 各エリアの情報をループ処理
        for area_data in area:
            area_name = area_data['area']['name']
            area_code = area_data['area']['code']
            weather_codes = area_data.get('weatherCodes', ['なし'])
            weathers = area_data.get('weathers', ['なし'])
            winds = area_data.get('winds', ['なし'])
            waves = area_data.get('waves', ['なし'])

            # データを1行ずつDBに挿入
            for i in range(len(time_defines)):
                # 行データを準備
                row_data = (
                    id,  # ID
                    children_code,  # 子供コード
                    area_name,  # 地域名
                    area_code,  # 地域コード
                    time_defines[i],  # 日付
                    weather_codes[i] if i < len(weather_codes) else 'なし',  # 天気コード
                    weathers[i] if i < len(weathers) else 'なし',  # 天気
                    winds[i] if i < len(winds) else 'なし',  # 風
                    waves[i] if i < len(waves) else 'なし'  # 波
                )

                # 重複を避けるために、まずそのデータが存在するか確認
                cur.execute(f"""
                SELECT COUNT(*) FROM "{table_name}" WHERE id = ? AND children_code = ? AND date = ?
                """, (id, children_code, time_defines[i]))
                count = cur.fetchone()[0]

                # 存在しなければデータを挿入
                if count == 0:
                    cur.execute(f"""
                    INSERT INTO "{table_name}" (id, children_code, area_name, area_code, date, weather_codes, weather, wind, wave)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                    """, row_data)

    except requests.exceptions.RequestException as e:  # リクエストエラーが発生した場合
        print(f"URL {url} からのデータ取得エラー: {e}")

# コミットして変更を保存
con.commit()

# 作成されたテーブルの一覧を表示
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()
print("作成された一覧テーブル:")
for table in tables:
    table_name = table[0]
    if table_name.startswith("weather_forecast_areas_"):
        print(f"テーブル名: {table_name}")
        
        # 各テーブルのデータを表示（最大10行のみ表示）
        cur.execute(f"SELECT * FROM {table_name} LIMIT 10;")
        rows = cur.fetchall()
        print(f"テーブル {table_name} のデータ:")
        for row in rows:
            print(row)
        print("-" * 50)

# DBへの接続を閉じる
con_area_info.close()
con.close()

print("データベースの作成とデータ挿入が完了しました。")


id: 1, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 011000
id: 2, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 012000
id: 3, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 013000
id: 4, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 014030
URL https://www.jma.go.jp/bosai/forecast/data/forecast/014030.json からのデータ取得エラー: 404 Client Error: Not Found for url: https://www.jma.go.jp/bosai/forecast/data/forecast/014030.json
id: 5, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 014100
id: 6, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 015000
id: 7, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 016000
id: 8, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_n